In [1]:
# !pip install git+https://github.com/crispitagorico/sigkernel.git
# !git clone 'https://github.com/ryanmccrickerd/rough_bergomi.git'

In [1]:
import numpy as np
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time
import seaborn as sns

from rbergomi import rBergomi_MC_pricer, rBergomi_sigkernel_pricer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# model parameters
T, a, xi, eta, rho = 1., -.4, .055, 1.9, -.9
n_increments       = 20
log_strike         = -5.  
payoff             = lambda x: max(np.exp(x) - np.exp(log_strike), 0.) # call
x_var              = 5.

# evaluation points
n_eval       = 100
t_inds_eval  = np.random.choice(n_increments, n_eval)
xs_eval      = generate_xs(xi, x_var, t_inds_eval)
paths_eval   = generate_theta_paths(t_inds_eval, n_increments, T, a)

# sigkernel PDE computation params
dyadic_order, max_batch = 2, 200

# error
error_fn, error_name, precisions = mse, 'MSE', [1e-2, 1e-3, 1e-4]
# error_fn, error_name, precisions = mae, 'MAE', [1e-1]

In [14]:
# ground truth prices
n_samples_MC_exact = 100000
mc_pricer_exact = rBergomi_MC_pricer(n_increments, n_samples_MC_exact, T, a, xi, eta, rho)
mc_prices_exact = mc_pricer_exact.fit_predict(t_inds_eval, xs_eval, paths_eval, payoff)

In [ ]:
# grid search for kernel parameters
m, n = 100, 80
sigmas = [1e0, 5e0, 1e1, 5e1, 1e2, 5e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6]
mse_score = 1e5
for sigma_t in sigmas:
    for sigma_x in sigmas:
        for sigma_sig in sigmas:
            sig_pricer = rBergomi_sigkernel_pricer(n_increments, x_var, m, n, T, a, xi, eta, rho, sigma_t, sigma_x, sigma_sig, dyadic_order, max_batch, device)
            sig_pricer.fit(payoff)
            sig_prices = sig_pricer.predict(t_inds_eval, xs_eval, paths_eval) 
            mse_score_pred = error_fn(mc_prices_exact, sig_prices)
            if mse_score_pred < mse_score: 
                print(f'sigma_t: {sigma_t}, sigma_x: {sigma_x}, sigma_sig: {sigma_sig}, MSE: {mse_score_pred}')
                mse_score = mse_score_pred
                sigma_t_best, sigma_x_best, sigma_sig_best = sigma_t, sigma_x, sigma_sig

sigma_t: 1.0, sigma_x: 1.0, sigma_sig: 1.0, MSE: 6319.552721846827
sigma_t: 1.0, sigma_x: 1.0, sigma_sig: 5.0, MSE: 191.96201808324523
sigma_t: 1.0, sigma_x: 1.0, sigma_sig: 10.0, MSE: 11.687660436187288
sigma_t: 1.0, sigma_x: 1.0, sigma_sig: 500.0, MSE: 11.642344228580605
sigma_t: 5.0, sigma_x: 1.0, sigma_sig: 50.0, MSE: 3.3375056567778576
sigma_t: 5.0, sigma_x: 1.0, sigma_sig: 1000.0, MSE: 0.23824869232453835
sigma_t: 50.0, sigma_x: 1.0, sigma_sig: 500.0, MSE: 0.09240869775684976
sigma_t: 50.0, sigma_x: 1.0, sigma_sig: 1000000.0, MSE: 0.05983267025482176
sigma_t: 50.0, sigma_x: 5.0, sigma_sig: 5000.0, MSE: 0.0566868360243358
sigma_t: 50.0, sigma_x: 5.0, sigma_sig: 500000.0, MSE: 0.04300174799222233
sigma_t: 100.0, sigma_x: 1.0, sigma_sig: 50000.0, MSE: 0.0006217557785437444
sigma_t: 100.0, sigma_x: 1.0, sigma_sig: 500000.0, MSE: 4.314902428246287e-05
sigma_t: 500.0, sigma_x: 1.0, sigma_sig: 50000.0, MSE: 2.2547088306385336e-05


In [ ]:
n_samples_mc, error_mc = 0, 1e9
m, n, error_sig        = 0, 0, 1e9

for precision in precisions:

    # MC prices
    while error_mc > precision:
        n_samples_mc += 500
        mc_pricer = rBergomi_MC_pricer(n_increments, n_samples_mc, T, a, xi, eta, rho)
        t0 = time.time()
        mc_prices = mc_pricer.fit_predict(t_inds_eval, xs_eval, paths_eval, payoff)
        t1 = time.time()
        error_mc  = error_fn(mc_prices, mc_prices_exact)

    print('Monte Carlo | %r: %2.5f | sample paths: %r | time: %2.4f sec' % (error_name, precision, n_samples_mc, t1-t0))
    
    # PPDE prices
    while error_sig > precision:
        m += 100
        n += 80
        sig_pricer = rBergomi_sigkernel_pricer(n_increments, x_var, m, n, T, a, xi, eta, rho, sigma_t_best, sigma_x_best, sigma_sig_best, dyadic_order, max_batch, device)
        sig_pricer.fit(payoff)
        t0 = time.time()
        sig_prices = sig_pricer.predict(t_inds_eval, xs_eval, paths_eval) 
        t1 = time.time()
        error_sig = error_fn(sig_prices, mc_prices_exact)
        torch.cuda.empty_cache()

    print('PPDE (sigkernel) | %r: %2.5f | collocation points: (%r,%r) | time: %2.4f sec \n' % (error_name, precision, m, n, (t1-t0)))
